In [4]:
import re
import os

# --- FUNCIONES DE ANONIMIZACIÓN (Sin cambios) ---

def anonimizar_patrones_especificos(texto):
    """
    Busca y reemplaza patrones de datos sensibles específicos dentro de un texto.
    Ideal para el cuerpo del informe.
    """
    # Reemplazar números de identificación (ej. formato RUT chileno)
    texto = re.sub(r'\b\d{1,2}\.?\d{3}\.?\d{3}-[\dkK]\b', '[ID_ANONIMIZADO]', texto)
    # Reemplazar fechas
    texto = re.sub(r'\b\d{1,2}(/|-|\s+de\s+)\w+(\s+de\s+|-|/)\d{2,4}\b', '[FECHA_ANONIMIZADA]', texto, flags=re.IGNORECASE)
    # Reemplazar nombres después de etiquetas comunes (ejemplo simple)
    texto = re.sub(r'(Paciente:|Nombre:|Dr\.|Dra\.:)\s*([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+(\s+[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)*)', r'\1 [NOMBRE_ANONIMIZADO]', texto, flags=re.IGNORECASE)
    return texto

def anonimizar_informe_por_bloques(texto_completo):
    """
    Anonimiza un informe dividiéndolo en encabezado, cuerpo y pie de página,
    usando "Dr." o "Atentamente" como marcador de fin.
    """
    marcador_inicio_cuerpo = r"Fecha Examen"
    
    # ANTES:
    # marcador_fin_cuerpo = r"Dr(\(a\))?\."

    # AHORA (Solución con marcador alternativo):
    # Busca "Dr(a)." O la palabra "Atentamente"
    marcador_fin_cuerpo = r"Dr(\(a\))?\.|\bAtentamente\b" # <-- MODIFICA ESTA LÍNEA

    # Usamos re.search para encontrar las posiciones de los marcadores
    match_inicio = re.search(marcador_inicio_cuerpo, texto_completo, re.IGNORECASE)
    match_fin = re.search(marcador_fin_cuerpo, texto_completo, re.IGNORECASE)

    # El resto de la función sigue exactamente igual...
    if match_inicio and match_fin:
        pos_inicio_cuerpo = match_inicio.start()
        pos_fin_cuerpo = match_fin.start()
        cuerpo = texto_completo[pos_inicio_cuerpo:pos_fin_cuerpo]
        
        encabezado_anonimizado = "[BLOQUE DE DATOS DEL PACIENTE ANONIMIZADO]\n" + "-"*40 + "\n"
        pie_pagina_anonimizado = "\n" + "-"*40 + "\n[BLOQUE DE FIRMA Y DATOS DEL MÉDICO ANONIMIZADO]"
        cuerpo_anonimizado = anonimizar_patrones_especificos(cuerpo)
        
        return encabezado_anonimizado + cuerpo_anonimizado + pie_pagina_anonimizado
    else:
        print(f"  - ADVERTENCIA: Marcadores no encontrados. Se aplicará anonimización de patrones a todo el archivo.")
        return anonimizar_patrones_especificos(texto_completo)
        
def anonimizar_estructura_completa(carpeta_raiz_entrada, carpeta_raiz_salida):
    """
    Anonimiza archivos .txt en una carpeta y todas sus subcarpetas,
    replicando la estructura en una carpeta de salida.
    """
    print(f"Iniciando anonimización recursiva desde: '{carpeta_raiz_entrada}'")
    archivos_procesados = 0

    # os.walk() recorre todo el árbol de directorios.
    for dirpath, _, filenames in os.walk(carpeta_raiz_entrada):
        
        # 1. Crear la estructura de carpetas correspondiente en la salida.
        ruta_salida_actual = dirpath.replace(carpeta_raiz_entrada, carpeta_raiz_salida, 1)
        if not os.path.exists(ruta_salida_actual):
            os.makedirs(ruta_salida_actual)
            print(f"Creando directorio: {ruta_salida_actual}")

        # 2. Procesar todos los archivos .txt en el directorio actual.
        for nombre_archivo in filenames:
            if nombre_archivo.lower().endswith(".txt"):
                
                ruta_archivo_entrada = os.path.join(dirpath, nombre_archivo)
                ruta_archivo_salida = os.path.join(ruta_salida_actual, nombre_archivo)
                
                print(f"  - Anonimizando: {nombre_archivo}")

                with open(ruta_archivo_entrada, 'r', encoding='utf-8') as f_in:
                    contenido_original = f_in.read()
                
                # Usamos la función de anonimización por bloques
                contenido_anonimizado = anonimizar_informe_por_bloques(contenido_original)
                
                with open(ruta_archivo_salida, 'w', encoding='utf-8') as f_out:
                    f_out.write(contenido_anonimizado)
                
                archivos_procesados += 1

    print(f"\n✅ ¡Proceso completado! Se anonimizaron {archivos_procesados} archivos.")
    print(f"Los resultados se guardaron en la estructura replicada dentro de: '{carpeta_raiz_salida}'")

# --- EJEMPLO DE USO ---

# Carpeta raíz que contiene los archivos de texto originales (con subcarpetas)
CARPETA_RAIZ_ENTRADA = 'Informes_Unicos'

# Carpeta raíz donde se guardará la nueva estructura con los archivos anonimizados
CARPETA_RAIZ_SALIDA = 'Informes_Anonimizados'

# Llamamos a la nueva función principal para iniciar el proceso
anonimizar_estructura_completa(CARPETA_RAIZ_ENTRADA, CARPETA_RAIZ_SALIDA)

Iniciando anonimización recursiva desde: 'Informes_Unicos'
Creando directorio: Informes_Anonimizados
Creando directorio: Informes_Anonimizados\criticos
  - Anonimizando: C__Users_sebas_AppData_Roaming_Carestream_RIS_11.3.0.25596_risappn_CSHRIS_WS_Reporting_R9276005832311_134027204311172981.txt
  - Anonimizando: C__Users_sebas_AppData_Roaming_Carestream_RIS_11.3.0.25596_risappn_CSHRIS_WS_Reporting_R9276006431888_134020288721232893.txt
  - Anonimizando: C__Users_sebas_AppData_Roaming_Carestream_RIS_11.3.0.25596_risappn_CSHRIS_WS_Reporting_R9276006613028_134015158506207456.txt
  - Anonimizando: C__Users_sebas_AppData_Roaming_Carestream_RIS_11.3.0.25596_risappn_CSHRIS_WS_Reporting_R9276009622794_134020288259811673.txt
  - Anonimizando: C__Users_sebas_AppData_Roaming_Carestream_RIS_11.3.0.25596_risappn_CSHRIS_WS_Reporting_R9276010524792_134027176755023957.txt
  - Anonimizando: C__Users_sebas_AppData_Roaming_Carestream_RIS_11.3.0.25596_risappn_CSHRIS_WS_Reporting_R9276010535053_1340202929539